![Py4Eng](../logo.png)

# Keras Functional API
## Yoav Ram

The [functional API](https://keras.io/guides/functional_api/) is an advanced Keras feature that is used for building non-sequential networks, such as ResNet, or networks made from mutiple nested models, such as GAN and autoencoders.

In the functional API, each layer is a function that operates on an input to produce an output. These inputs and outputs are *symbolic* and do not contain any data when we set the network up. They will only contain data during training or predicting.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

import jax
import keras
print('Keras:', keras.__version__, 'backend:', keras.backend.backend(), jax.default_backend())

Keras: 3.8.0 backend: jax cpu


In [3]:
def display_image(im):
    fig, ax = plt.subplots()
    ax.imshow(im, cmap='gray_r')
    ax.set_xticks([])
    ax.set_yticks([])    

In [4]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
nsamples, width, height = X_train.shape
X_train = (X_train/255).astype('float32')
X_test = (X_test/255).astype('float32')

In [5]:
Y_train = keras.utils.to_categorical(Y_train)
Y_test = keras.utils.to_categorical(Y_test)
ncats = Y_test.shape[1]

# FFN with `Sequential`

This is the two-layers simple feed forward network we implemented already.

In [11]:
model = keras.models.Sequential()

model.add(
    keras.Input((width, height)))
model.add(
    keras.layers.Flatten())
model.add(
    keras.layers.Dense(100, activation='relu'))
model.add(
    keras.layers.Dropout(rate=0.5))
model.add(
    keras.layers.Dense(ncats, activation='softmax'))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [13]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Epoch 1/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7785 - loss: 0.7237 - val_accuracy: 0.9436 - val_loss: 0.1882
Epoch 2/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9191 - loss: 0.2768 - val_accuracy: 0.9565 - val_loss: 0.1465
Epoch 3/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9353 - loss: 0.2176 - val_accuracy: 0.9621 - val_loss: 0.1298
Epoch 4/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9401 - loss: 0.2025 - val_accuracy: 0.9670 - val_loss: 0.1111
Epoch 5/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9457 - loss: 0.1812 - val_accuracy: 0.9693 - val_loss: 0.1068


# FFN with functional API 

We now implement it with the functional API.

In this API, each layer is a function that works on some input tensor. A tensor is basically a placeholder for future data (array).

We first create the layers, then apply them to the tensors. 
The model is then created using the input and the output tensors.

In [14]:
flatten_layer = keras.layers.Flatten()
hidden_layer = keras.layers.Dense(100, activation="relu")
dropout_layer = keras.layers.Dropout(rate=0.5)
softmax_layer = keras.layers.Dense(ncats, activation="softmax")

x = keras.Input((width, height))
x_flat = flatten_layer(x)
h = hidden_layer(x_flat)
h_drop = dropout_layer(h)
yhat = softmax_layer(h_drop)

model = keras.Model(inputs=x, outputs=yhat)
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 28, 28)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

The rest is per usual - compile and fit.

In [15]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [16]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Epoch 1/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7813 - loss: 0.7120 - val_accuracy: 0.9411 - val_loss: 0.2008
Epoch 2/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9196 - loss: 0.2780 - val_accuracy: 0.9548 - val_loss: 0.1535
Epoch 3/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9337 - loss: 0.2227 - val_accuracy: 0.9605 - val_loss: 0.1372
Epoch 4/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9408 - loss: 0.1998 - val_accuracy: 0.9656 - val_loss: 0.1187
Epoch 5/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9450 - loss: 0.1884 - val_accuracy: 0.9680 - val_loss: 0.1073


# Residual connections

We can use the functional API to build a network with residual connections. 
Note: It's not really comparable to the previous network because the dense layer output has to be modifier to 784 units.

In [26]:
x = keras.Input((width, height))
x_flat = keras.layers.Flatten()(x)
h = keras.layers.Dense(width * height, activation="relu")(x_flat)
x_add_h = keras.layers.Add()([x_flat, h]) # Add x and h
x_add_h = keras.layers.Dropout(rate=0.5)(x_add_h)
yhat = keras.layers.Dense(ncats, activation="softmax")(x_add_h)

model = keras.Model(inputs=x, outputs=yhat)
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 784)       │          0 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 784)       │    615,440 │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 784)       │          0 │ flatten_9[0][0],  │
│                     │                   │            │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 784)       │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 10)        │      7,850 │ dropout_6[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 623,290 (2.38 MB)

 Trainable params: 623,290 (2.38 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [28]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Epoch 1/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8573 - loss: 0.4634 - val_accuracy: 0.9631 - val_loss: 0.1226
Epoch 2/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9555 - loss: 0.1492 - val_accuracy: 0.9712 - val_loss: 0.0951
Epoch 3/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9655 - loss: 0.1098 - val_accuracy: 0.9747 - val_loss: 0.0829
Epoch 4/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9718 - loss: 0.0884 - val_accuracy: 0.9783 - val_loss: 0.0692
Epoch 5/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9752 - loss: 0.0765 - val_accuracy: 0.9772 - val_loss: 0.0755


# Exercise: CNN

Implement a CNN using the functional API. See the sequenctial example below (copied from [the CNN session](K_CNN.ipynb)).

[solution](../solutions/functional_keras.py)

In [31]:
# Sequential example
model = keras.models.Sequential()

model.add(
    keras.Input((width, height)))
model.add(
    keras.layers.Reshape(target_shape=(width, height, 1)))
model.add(
    keras.layers.Conv2D(32, (5, 5), activation='relu'))
model.add(
    keras.layers.MaxPool2D())
model.add(
    keras.layers.Conv2D(64, (5, 5), activation='relu'))
model.add(
    keras.layers.MaxPool2D())
model.add(
    keras.layers.Flatten())
model.add(
    keras.layers.Dense(1024, activation='relu'))
model.add(
    keras.layers.Dropout(rate=0.5))
model.add(
    keras.layers.Dense(ncats, activation='softmax'))

In [28]:
# Functional 
# your code here

################

In [32]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [33]:
history = model.fit(x=X_train, y=Y_train, batch_size=50, epochs=5, 
    validation_data=(X_test, Y_test),
).history

Epoch 1/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 72s 60ms/step - accuracy: 0.9110 - loss: 0.2890 - val_accuracy: 0.9890 - val_loss: 0.0377
Epoch 2/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 77s 64ms/step - accuracy: 0.9865 - loss: 0.0434 - val_accuracy: 0.9892 - val_loss: 0.0331
Epoch 3/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 79s 66ms/step - accuracy: 0.9909 - loss: 0.0297 - val_accuracy: 0.9930 - val_loss: 0.0221
Epoch 4/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 79s 66ms/step - accuracy: 0.9930 - loss: 0.0233 - val_accuracy: 0.9916 - val_loss: 0.0272
Epoch 5/5
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 114s 95ms/step - accuracy: 0.9939 - loss: 0.0206 - val_accuracy: 0.9917 - val_loss: 0.0239


# References

- [Keras functional API](https://keras.io/guides/functional_api/)

# Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)